In [ ]:
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup, SoupStrainer
import os
import multiprocessing


In [ ]:
df = pd.read_table("./temp.tsv")
df


In [ ]:
ExpID = [link.split("/")[5] for link in df.iloc[:, 1]]
ExpTitle = [title for title in df.iloc[:, 2]]
# ExpID


In [ ]:
urlRoot = "https://www.ebi.ac.uk/gxa/experiments-content/"
resources = [
    "/resources/ExperimentDownloadSupplier.RnaSeqDifferential/tsv",
    "/resources/DifferentialSecondaryDataFiles.RnaSeq/analytics",
    "/resources/DifferentialSecondaryDataFiles.RnaSeq/raw-counts",
    "/resources/ExperimentDesignFile.RnaSeq/experiment-design",
    "/resources/ExperimentDownloadSupplier.Microarray/query-results",
    "/resources/DifferentialSecondaryDataFiles.Microarray/analytics",
    "/resources/DifferentialSecondaryDataFiles.Microarray/normalized-expressions",
    "/resources/ExperimentDesignFile.Microarray/experiment-design",
]

if not os.path.isdir("data"):
    os.mkdir("data")

os.chdir("./data")
for ID, title in zip(ExpID, ExpTitle):
    if not os.path.isdir(f"{ID}"):
        os.mkdir(f"{ID}")
    os.chdir(f"{ID}")
    for url in [urlRoot + ID + resource for resource in resources]:
        os.system(f'curl -x 16 "{url}"')
    with open(f"Info.log", "w") as f:
        f.write(title)
    os.chdir("..")

os.chdir("..")


In [ ]:
urlRoot = "https://www.ebi.ac.uk/gxa/experiments-content/"
resources = [
    "/resources/ExperimentDownloadSupplier.RnaSeqDifferential/tsv",
    "/resources/DifferentialSecondaryDataFiles.RnaSeq/analytics",
    "/resources/DifferentialSecondaryDataFiles.RnaSeq/raw-counts",
    "/resources/ExperimentDesignFile.RnaSeq/experiment-design",
    "/resources/ExperimentDownloadSupplier.Microarray/query-results",
    "/resources/DifferentialSecondaryDataFiles.Microarray/analytics",
    "/resources/DifferentialSecondaryDataFiles.Microarray/normalized-expressions",
    "/resources/ExperimentDesignFile.Microarray/experiment-design",
]
print(
    "\n".join(
        ["\n".join([urlRoot + ID + resource for resource in resources]) for ID in ExpID]
    )
)


In [ ]:
def download(url, filename, savepath, alternate):
    blockSize = 1024 * 1024
    try:
        resp = urllib.request.urlopen(url)
        with open(f"{savepath}/{filename}", "wb") as f:
            while True:
                buffer = resp.read(blockSize)
                if not buffer:
                    break
                f.write(buffer)
    except Exception:
        if alternate:
            try:
                resp = urllib.request.urlopen(alternate)
                with open(f"{savepath}/{filename}", "wb") as f:
                    while True:
                        buffer = resp.read(blockSize)
                        if not buffer:
                            break
                        f.write(buffer)
            except Exception:
                print(f"{filename} download failed")
                return False
        else:
            return False
    return True


def downloadExperimentData(ID):
    if not os.path.isdir(f"data/{ID}"):
        os.mkdir(f"data/{ID}")
    status = {
        "ExpressionResult.tsv": True,
        "Analytics.tsv": True,
        "RawCounts-NormExpression.tsv": True,
        "Metadata.tsv": True,
    }
    urls = {
        "ExpressionResult.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDownloadSupplier.RnaSeqDifferential/tsv",
        "Analytics.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.RnaSeq/analytics",
        "RawCounts-NormExpression.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.RnaSeq/raw-counts",
        "Metadata.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDesignFile.RnaSeq/experiment-design",
    }
    alt = {
        "ExpressionResult.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDownloadSupplier.Microarray/query-results",
        "Analytics.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.Microarray/analytics",
        "RawCounts-NormExpression.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.Microarray/normalized-expressions",
        "Metadata.tsv": f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDesignFile.Microarray/experiment-design",
    }
    processPool = []
    for fileName in status.keys():
        if not os.path.isfile(f"data/{ID}/{fileName}"):
            processPool.append(
                multiprocessing.Process(
                    target=download,
                    args=(urls[fileName], fileName, f"data/{ID}", alt[fileName]),
                )
            )
    for proc in processPool:
        proc.start()
    for proc in processPool:
        proc.join()
    return


if not os.path.isdir("data"):
    os.mkdir("data")
for ID, title in zip(ExpID, ExpTitle):
    print(f"https://www.ebi.ac.uk/gxa/experiments/{ID}/Downloads")
    print(f"{ID} started downloading")
    with open(f"data/{ID}/Info.log", "w") as f:
        f.write(title)
    downloadExperimentData(ID)


In [ ]:
resp = urllib.request.urlopen(df.iloc[0, 1])


In [ ]:
soup = BeautifulSoup(
    markup=resp, features="html.parser", from_encoding=resp.info().get_param("charset")
)


In [ ]:
if not os.path.isdir("data"):
    os.mkdir("data")
trash = [os.mkdir(f"data/{ID}") for ID in ExpID if not os.path.isdir(f"data/{ID}")]


In [ ]:
def downloadFile(url, filename, savepath):
    resp = urllib.request.urlopen(url)
    blockSize = 1024 * 32
    with open(f"{savepath}/{filename}", "wb") as f:
        while True:
            buffer = resp.read(blockSize)
            if not buffer:
                break
            f.write(buffer)
    return


for ID in ExpID:
    if not os.path.isdir(f"data/{ID}"):
        os.mkdir(f"data/{ID}")
    try:
        downloadFile(
            f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDownloadSupplier.RnaSeqDifferential/tsv",
            "ExpressionResult.tsv",
            f"data/{ID}",
        )
    except Exception:
        print(f"{ID}'s ExpressionResult.tsv download failed")
    try:
        downloadFile(
            f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.RnaSeq/analytics",
            "Analytics.tsv",
            f"data/{ID}",
        )
    except Exception:
        print(f"{ID}'s Analytics.tsv download failed")
    try:
        downloadFile(
            f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/DifferentialSecondaryDataFiles.RnaSeq/raw-counts",
            "RawCounts.tsv",
            f"data/{ID}",
        )
    except Exception:
        print(f"{ID}'s RawCounts.tsv download failed")
    try:
        downloadFile(
            f"https://www.ebi.ac.uk/gxa/experiments-content/{ID}/resources/ExperimentDesignFile.RnaSeq/experiment-design",
            "Metadata.tsv",
            f"data/{ID}",
        )
    except Exception:
        print(f"{ID}'s Metadata.tsv download failed")
